## ⚙️ Конфигурация

In [ ]:
# =============================================================================
# КОНФИГУРАЦИЯ - измените параметры под свои нужды
# =============================================================================

# Режим демо (True = быстрое выполнение ~15 мин, False = полное обучение ~50 мин)
DEMO_MODE = True

# Параметры датасета
IMAGE_SIZE = 64          # Размер изображений (64x64 для CelebA)
MAX_IMAGES = 20000 if DEMO_MODE else None  # Ограничение количества изображений

# Параметры обучения
BATCH_SIZE = 64
NUM_EPOCHS = 15 if DEMO_MODE else 30
LEARNING_RATE = 0.0002
LATENT_DIM = 100         # Размерность латентного пространства

# Веса loss функций
TIGAS_LOSS_WEIGHT = 0.1  # Вес TIGAS loss в общем loss генератора

print(f"{'='*50}")
print(f"Режим: {'ДЕМО (быстрый)' if DEMO_MODE else 'ПОЛНЫЙ'}")
print(f"Эпох: {NUM_EPOCHS}")
print(f"Изображений: {MAX_IMAGES if MAX_IMAGES else 'все'}")
print(f"{'='*50}")

## 📦 1. Установка зависимостей

In [ ]:
# Проверяем наличие GPU
import subprocess
result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], 
                       capture_output=True, text=True)
if result.returncode == 0:
    print(f"✅ GPU обнаружен: {result.stdout.strip()}")
else:
    print("❌ GPU не обнаружен! Переключите runtime на GPU.")
    print("   Runtime → Change runtime type → T4 GPU")

In [ ]:
%%capture
# Установка PyTorch с CUDA (Colab обычно уже имеет, но обновим для уверенности)
!pip install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu121

# Установка TIGAS с TestPyPI
!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ tigas-metric

# Дополнительные библиотеки для визуализации
!pip install gdown

In [ ]:
# Проверка установки
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

from tigas import TIGAS, __version__ as tigas_version
print(f"\nTIGAS version: {tigas_version}")
print("\n✅ Все зависимости установлены!")

## 🔍 2. Демонстрация инференса TIGAS

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
import os

# Инициализация TIGAS (автозагрузка модели с HuggingFace)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tigas = TIGAS(auto_download=True, device=device)
print(f"\n✅ TIGAS загружен на {device}")

In [ ]:
# Скачиваем примеры изображений для демонстрации
os.makedirs('demo_images', exist_ok=True)

# Реальные фотографии (с Unsplash - бесплатные)
real_images_urls = [
    ('https://images.unsplash.com/photo-1507003211169-0a1dd7228f2d?w=256&h=256&fit=crop', 'real_1.jpg'),
    ('https://images.unsplash.com/photo-1494790108377-be9c29b29330?w=256&h=256&fit=crop', 'real_2.jpg'),
    ('https://images.unsplash.com/photo-1500648767791-00dcc994a43e?w=256&h=256&fit=crop', 'real_3.jpg'),
]

print("Загрузка примеров реальных изображений...")
for url, filename in real_images_urls:
    path = f'demo_images/{filename}'
    if not os.path.exists(path):
        try:
            urllib.request.urlretrieve(url, path)
            print(f"  ✓ {filename}")
        except Exception as e:
            print(f"  ✗ {filename}: {e}")

In [ ]:
# Создаём "фейковые" изображения (шум и простые паттерны)
def create_fake_images():
    fakes = []
    
    # 1. Случайный шум
    noise = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
    fakes.append(('Случайный шум', noise))
    
    # 2. Градиент
    gradient = np.zeros((256, 256, 3), dtype=np.uint8)
    for i in range(256):
        gradient[i, :, 0] = i
        gradient[:, i, 1] = i
    fakes.append(('Градиент', gradient))
    
    # 3. Шахматный паттерн
    checker = np.zeros((256, 256, 3), dtype=np.uint8)
    for i in range(256):
        for j in range(256):
            if (i // 32 + j // 32) % 2 == 0:
                checker[i, j] = [255, 255, 255]
    fakes.append(('Шахматный паттерн', checker))
    
    return fakes

fake_images = create_fake_images()
print(f"✅ Создано {len(fake_images)} синтетических изображений")

In [ ]:
# Оценка реальных изображений
print("\n" + "="*50)
print("ОЦЕНКА РЕАЛЬНЫХ ИЗОБРАЖЕНИЙ")
print("="*50)

real_scores = []
real_imgs = []

for url, filename in real_images_urls:
    path = f'demo_images/{filename}'
    if os.path.exists(path):
        score = tigas(path)
        real_scores.append(score.item())
        real_imgs.append(Image.open(path))
        print(f"{filename}: TIGAS = {score.item():.4f}")

if real_scores:
    print(f"\nСреднее для реальных: {np.mean(real_scores):.4f}")

In [ ]:
# Оценка синтетических изображений
print("\n" + "="*50)
print("ОЦЕНКА СИНТЕТИЧЕСКИХ ИЗОБРАЖЕНИЙ")
print("="*50)

fake_scores = []

for name, img_array in fake_images:
    img = Image.fromarray(img_array)
    score = tigas(img)
    fake_scores.append(score.item())
    print(f"{name}: TIGAS = {score.item():.4f}")

print(f"\nСреднее для синтетических: {np.mean(fake_scores):.4f}")

In [ ]:
# Визуализация результатов
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Реальные изображения
for i, (img, score) in enumerate(zip(real_imgs[:3], real_scores[:3])):
    axes[0, i].imshow(img)
    axes[0, i].set_title(f'Реальное\nTIGAS: {score:.3f}', fontsize=12)
    axes[0, i].axis('off')
    # Цветовая рамка
    for spine in axes[0, i].spines.values():
        spine.set_edgecolor('green')
        spine.set_linewidth(3)

# Синтетические изображения
for i, ((name, img_array), score) in enumerate(zip(fake_images[:3], fake_scores[:3])):
    axes[1, i].imshow(img_array)
    axes[1, i].set_title(f'{name}\nTIGAS: {score:.3f}', fontsize=12)
    axes[1, i].axis('off')
    for spine in axes[1, i].spines.values():
        spine.set_edgecolor('red')
        spine.set_linewidth(3)

plt.suptitle('TIGAS: Оценка реальных vs синтетических изображений\n(Выше = более реалистично)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Вывод: TIGAS даёт высокие оценки реальным фото и низкие — синтетическим паттернам")

## 🎨 3. Подготовка данных для обучения GAN

In [ ]:
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, Subset

# Трансформации для CelebA
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Нормализация в [-1, 1]
])

print(f"Загрузка CelebA датасета (это может занять несколько минут)...")

In [ ]:
# Загрузка CelebA
# В Colab датасет скачивается автоматически
try:
    dataset = datasets.CelebA(
        root='./data',
        split='train',
        transform=transform,
        download=True
    )
except Exception as e:
    print(f"Ошибка загрузки CelebA: {e}")
    print("\nПопробуем альтернативный метод через gdown...")
    
    # Альтернативная загрузка через gdown
    !mkdir -p ./data/celeba
    !gdown --fuzzy "https://drive.google.com/uc?id=0B7EVK8r0v71pZjFTYXZWM3FlRnM" -O ./data/celeba/img_align_celeba.zip
    !unzip -q ./data/celeba/img_align_celeba.zip -d ./data/celeba/
    
    # Используем ImageFolder как запасной вариант
    dataset = datasets.ImageFolder(
        root='./data/celeba',
        transform=transform
    )

In [ ]:
# Ограничиваем количество изображений для демо режима
if MAX_IMAGES and len(dataset) > MAX_IMAGES:
    indices = torch.randperm(len(dataset))[:MAX_IMAGES]
    dataset = Subset(dataset, indices)
    print(f"✅ Используется подмножество: {len(dataset)} изображений")
else:
    print(f"✅ Используется весь датасет: {len(dataset)} изображений")

# Создаём DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

print(f"Батчей в эпохе: {len(dataloader)}")

In [ ]:
# Визуализация примеров из датасета
real_batch = next(iter(dataloader))
if isinstance(real_batch, (list, tuple)):
    real_batch = real_batch[0]  # CelebA возвращает (images, attributes)

plt.figure(figsize=(12, 4))
for i in range(8):
    plt.subplot(2, 4, i + 1)
    img = real_batch[i].permute(1, 2, 0).numpy()
    img = (img + 1) / 2  # Денормализация из [-1,1] в [0,1]
    plt.imshow(np.clip(img, 0, 1))
    plt.axis('off')
plt.suptitle('Примеры из CelebA датасета', fontsize=14)
plt.tight_layout()
plt.show()

## 🏗️ 4. Архитектура DCGAN

In [ ]:
import torch.nn as nn

# Инициализация весов
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


class Generator(nn.Module):
    """DCGAN Generator: z (latent) → 64x64 RGB image"""
    
    def __init__(self, latent_dim=100, ngf=64):
        super().__init__()
        self.main = nn.Sequential(
            # Input: latent_dim x 1 x 1
            nn.ConvTranspose2d(latent_dim, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # State: (ngf*8) x 4 x 4
            
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # State: (ngf*4) x 8 x 8
            
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # State: (ngf*2) x 16 x 16
            
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # State: ngf x 32 x 32
            
            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: 3 x 64 x 64
        )
    
    def forward(self, z):
        return self.main(z)


class Discriminator(nn.Module):
    """DCGAN Discriminator: 64x64 RGB image → real/fake score"""
    
    def __init__(self, ndf=64):
        super().__init__()
        self.main = nn.Sequential(
            # Input: 3 x 64 x 64
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # State: ndf x 32 x 32
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*2) x 16 x 16
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*4) x 8 x 8
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*8) x 4 x 4
            
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            # Output: 1 x 1 x 1
        )
    
    def forward(self, x):
        return self.main(x).view(-1, 1).squeeze(1)


# Создаём модели
generator = Generator(LATENT_DIM).to(device)
discriminator = Discriminator().to(device)

# Инициализация весов
generator.apply(weights_init)
discriminator.apply(weights_init)

print(f"Generator параметров: {sum(p.numel() for p in generator.parameters()):,}")
print(f"Discriminator параметров: {sum(p.numel() for p in discriminator.parameters()):,}")

In [ ]:
# Тестовая генерация (до обучения)
with torch.no_grad():
    z = torch.randn(8, LATENT_DIM, 1, 1, device=device)
    fake_images_test = generator(z)

plt.figure(figsize=(12, 3))
for i in range(8):
    plt.subplot(1, 8, i + 1)
    img = fake_images_test[i].cpu().permute(1, 2, 0).numpy()
    img = (img + 1) / 2
    plt.imshow(np.clip(img, 0, 1))
    plt.axis('off')
plt.suptitle('Генерации ДО обучения (случайные веса)', fontsize=14)
plt.tight_layout()
plt.show()

## 🚀 5. Обучение GAN с TIGAS Loss

In [ ]:
import torch.optim as optim
from tqdm.auto import tqdm
import torchvision.transforms.functional as TF

# Loss и оптимизаторы
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

# Фиксированный шум для визуализации прогресса
fixed_noise = torch.randn(64, LATENT_DIM, 1, 1, device=device)

# Метки
real_label = 1.0
fake_label = 0.0

# Функция для подготовки изображений для TIGAS
# TIGAS ожидает нормализацию [-1, 1] и размер 256x256
def prepare_for_tigas(images):
    """Подготовка изображений для TIGAS (resize до 256x256)"""
    # images уже в [-1, 1], нужен только resize
    return TF.resize(images, [256, 256], antialias=True)

In [ ]:
# История обучения
history = {
    'D_loss': [],
    'G_loss': [],
    'G_loss_adv': [],
    'G_loss_tigas': [],
    'D_real': [],
    'D_fake': [],
    'tigas_score_fake': [],
    'tigas_score_real': []
}

# Сохранение генераций по эпохам
generated_images_history = []

print(f"\n{'='*60}")
print(f"НАЧАЛО ОБУЧЕНИЯ")
print(f"Эпох: {NUM_EPOCHS}, Батчей: {len(dataloader)}, TIGAS weight: {TIGAS_LOSS_WEIGHT}")
print(f"{'='*60}\n")

In [ ]:
# Основной цикл обучения
for epoch in range(NUM_EPOCHS):
    epoch_D_loss = []
    epoch_G_loss = []
    epoch_G_adv = []
    epoch_G_tigas = []
    epoch_D_real = []
    epoch_D_fake = []
    epoch_tigas_fake = []
    epoch_tigas_real = []
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS}')
    
    for i, batch in enumerate(pbar):
        # Получаем реальные изображения
        if isinstance(batch, (list, tuple)):
            real_images = batch[0].to(device)
        else:
            real_images = batch.to(device)
        
        batch_size = real_images.size(0)
        
        # =====================
        # Обучение Discriminator
        # =====================
        discriminator.zero_grad()
        
        # Реальные изображения
        label_real = torch.full((batch_size,), real_label, device=device, dtype=torch.float)
        output_real = discriminator(real_images)
        loss_D_real = criterion(output_real, label_real)
        loss_D_real.backward()
        D_real = output_real.mean().item()
        
        # Фейковые изображения
        noise = torch.randn(batch_size, LATENT_DIM, 1, 1, device=device)
        fake_images = generator(noise)
        label_fake = torch.full((batch_size,), fake_label, device=device, dtype=torch.float)
        output_fake = discriminator(fake_images.detach())
        loss_D_fake = criterion(output_fake, label_fake)
        loss_D_fake.backward()
        D_fake = output_fake.mean().item()
        
        loss_D = loss_D_real + loss_D_fake
        optimizer_D.step()
        
        # =====================
        # Обучение Generator
        # =====================
        generator.zero_grad()
        
        # Adversarial loss (обмануть discriminator)
        output = discriminator(fake_images)
        loss_G_adv = criterion(output, label_real)  # Хотим, чтобы D думал, что это реальные
        
        # TIGAS loss (максимизировать реалистичность по TIGAS)
        fake_for_tigas = prepare_for_tigas(fake_images)
        with torch.no_grad():  # TIGAS только для оценки, не для градиентов через него
            tigas_scores_fake = tigas(fake_for_tigas)
        
        # Для обучения: хотим максимизировать TIGAS score → минимизируем (1 - score)
        # Но TIGAS должен быть дифференцируемым для этого
        # Здесь мы используем TIGAS как дополнительный сигнал
        fake_for_tigas_grad = prepare_for_tigas(fake_images)
        tigas_scores_grad = tigas(fake_for_tigas_grad)
        loss_G_tigas = (1.0 - tigas_scores_grad).mean()  # Минимизируем (1 - score)
        
        # Полный loss генератора
        loss_G = loss_G_adv + TIGAS_LOSS_WEIGHT * loss_G_tigas
        loss_G.backward()
        optimizer_G.step()
        
        # Оценка TIGAS на реальных (для мониторинга)
        if i % 50 == 0:
            with torch.no_grad():
                real_for_tigas = prepare_for_tigas(real_images[:8])
                tigas_scores_real = tigas(real_for_tigas).mean().item()
        else:
            tigas_scores_real = epoch_tigas_real[-1] if epoch_tigas_real else 0.5
        
        # Сохраняем метрики
        epoch_D_loss.append(loss_D.item())
        epoch_G_loss.append(loss_G.item())
        epoch_G_adv.append(loss_G_adv.item())
        epoch_G_tigas.append(loss_G_tigas.item())
        epoch_D_real.append(D_real)
        epoch_D_fake.append(D_fake)
        epoch_tigas_fake.append(tigas_scores_fake.mean().item())
        epoch_tigas_real.append(tigas_scores_real)
        
        # Обновляем progress bar
        pbar.set_postfix({
            'D_loss': f'{loss_D.item():.3f}',
            'G_loss': f'{loss_G.item():.3f}',
            'TIGAS': f'{tigas_scores_fake.mean().item():.3f}'
        })
    
    # Сохраняем средние метрики за эпоху
    history['D_loss'].append(np.mean(epoch_D_loss))
    history['G_loss'].append(np.mean(epoch_G_loss))
    history['G_loss_adv'].append(np.mean(epoch_G_adv))
    history['G_loss_tigas'].append(np.mean(epoch_G_tigas))
    history['D_real'].append(np.mean(epoch_D_real))
    history['D_fake'].append(np.mean(epoch_D_fake))
    history['tigas_score_fake'].append(np.mean(epoch_tigas_fake))
    history['tigas_score_real'].append(np.mean(epoch_tigas_real))
    
    # Генерируем примеры для визуализации
    with torch.no_grad():
        fake_samples = generator(fixed_noise).cpu()
        generated_images_history.append(fake_samples[:16])
    
    # Вывод статистики
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}:")
    print(f"  D_loss: {history['D_loss'][-1]:.4f} | G_loss: {history['G_loss'][-1]:.4f}")
    print(f"  D(real): {history['D_real'][-1]:.4f} | D(fake): {history['D_fake'][-1]:.4f}")
    print(f"  TIGAS(fake): {history['tigas_score_fake'][-1]:.4f} | TIGAS(real): {history['tigas_score_real'][-1]:.4f}")

print(f"\n{'='*60}")
print("ОБУЧЕНИЕ ЗАВЕРШЕНО!")
print(f"{'='*60}")

## 📊 6. Визуализация результатов

In [ ]:
# Графики обучения
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

epochs_range = range(1, NUM_EPOCHS + 1)

# 1. Loss
axes[0, 0].plot(epochs_range, history['D_loss'], 'b-', label='Discriminator', linewidth=2)
axes[0, 0].plot(epochs_range, history['G_loss'], 'r-', label='Generator', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Generator Loss компоненты
axes[0, 1].plot(epochs_range, history['G_loss_adv'], 'g-', label='Adversarial', linewidth=2)
axes[0, 1].plot(epochs_range, history['G_loss_tigas'], 'm-', label=f'TIGAS (×{TIGAS_LOSS_WEIGHT})', linewidth=2)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Generator Loss Components')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Discriminator выходы
axes[1, 0].plot(epochs_range, history['D_real'], 'g-', label='D(real)', linewidth=2)
axes[1, 0].plot(epochs_range, history['D_fake'], 'r-', label='D(fake)', linewidth=2)
axes[1, 0].axhline(y=0.5, color='k', linestyle='--', alpha=0.5)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Discriminator Output')
axes[1, 0].set_title('Discriminator Predictions')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylim([0, 1])

# 4. TIGAS Score
axes[1, 1].plot(epochs_range, history['tigas_score_real'], 'g-', label='TIGAS(real)', linewidth=2)
axes[1, 1].plot(epochs_range, history['tigas_score_fake'], 'r-', label='TIGAS(fake)', linewidth=2)
axes[1, 1].fill_between(epochs_range, history['tigas_score_fake'], alpha=0.3, color='red')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('TIGAS Score')
axes[1, 1].set_title('TIGAS Score Evolution')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim([0, 1])

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 TIGAS score сгенерированных изображений растёт в процессе обучения!")

In [ ]:
# Эволюция генерации по эпохам
num_snapshots = min(5, len(generated_images_history))
snapshot_indices = np.linspace(0, len(generated_images_history) - 1, num_snapshots, dtype=int)

fig, axes = plt.subplots(num_snapshots, 8, figsize=(16, num_snapshots * 2))

for row, idx in enumerate(snapshot_indices):
    epoch_num = idx + 1
    images = generated_images_history[idx]
    
    for col in range(8):
        ax = axes[row, col] if num_snapshots > 1 else axes[col]
        img = images[col].permute(1, 2, 0).numpy()
        img = (img + 1) / 2
        ax.imshow(np.clip(img, 0, 1))
        ax.axis('off')
        if col == 0:
            ax.set_ylabel(f'Epoch {epoch_num}', fontsize=12, rotation=0, labelpad=40)

plt.suptitle('Эволюция генерации по эпохам', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('generation_evolution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Финальные генерации с TIGAS оценкой
with torch.no_grad():
    z = torch.randn(16, LATENT_DIM, 1, 1, device=device)
    final_fakes = generator(z)
    
    # Оцениваем каждое изображение
    final_fakes_256 = prepare_for_tigas(final_fakes)
    final_scores = tigas(final_fakes_256)

fig, axes = plt.subplots(4, 4, figsize=(12, 12))

for i, ax in enumerate(axes.flat):
    img = final_fakes[i].cpu().permute(1, 2, 0).numpy()
    img = (img + 1) / 2
    ax.imshow(np.clip(img, 0, 1))
    score = final_scores[i].item()
    color = 'green' if score > 0.5 else 'orange' if score > 0.3 else 'red'
    ax.set_title(f'TIGAS: {score:.3f}', fontsize=11, color=color, fontweight='bold')
    ax.axis('off')

plt.suptitle(f'Финальные генерации (средний TIGAS: {final_scores.mean().item():.3f})', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('final_generations.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Статистика TIGAS для сгенерированных изображений:")
print(f"   Среднее: {final_scores.mean().item():.4f}")
print(f"   Мин: {final_scores.min().item():.4f}")
print(f"   Макс: {final_scores.max().item():.4f}")
print(f"   Std: {final_scores.std().item():.4f}")

In [ ]:
# Сравнение: реальные vs сгенерированные
fig, axes = plt.subplots(2, 8, figsize=(16, 5))

# Реальные
real_sample = next(iter(dataloader))
if isinstance(real_sample, (list, tuple)):
    real_sample = real_sample[0]
real_sample = real_sample[:8].to(device)

with torch.no_grad():
    real_256 = prepare_for_tigas(real_sample)
    real_tigas = tigas(real_256)

for i in range(8):
    img = real_sample[i].cpu().permute(1, 2, 0).numpy()
    img = (img + 1) / 2
    axes[0, i].imshow(np.clip(img, 0, 1))
    axes[0, i].set_title(f'{real_tigas[i].item():.3f}', fontsize=10, color='green')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_ylabel('Реальные', fontsize=12, rotation=0, labelpad=50)

# Сгенерированные
for i in range(8):
    img = final_fakes[i].cpu().permute(1, 2, 0).numpy()
    img = (img + 1) / 2
    axes[1, i].imshow(np.clip(img, 0, 1))
    axes[1, i].set_title(f'{final_scores[i].item():.3f}', fontsize=10, color='blue')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_ylabel('Сгенер.', fontsize=12, rotation=0, labelpad=50)

plt.suptitle('Сравнение: Реальные vs Сгенерированные (числа = TIGAS score)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📈 Сравнение средних TIGAS:")
print(f"   Реальные: {real_tigas.mean().item():.4f}")
print(f"   Сгенерированные: {final_scores.mean().item():.4f}")
print(f"   Разница: {real_tigas.mean().item() - final_scores.mean().item():.4f}")

## 📝 7. Выводы

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║                        ИТОГИ ЭКСПЕРИМЕНТА                        ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  ✅ TIGAS успешно различает реальные и синтетические изображения ║
║                                                                  ║
║  ✅ TIGAS можно использовать как дифференцируемый loss           ║
║     для обучения генеративных моделей                            ║
║                                                                  ║
║  ✅ TIGAS score сгенерированных изображений растёт               ║
║     в процессе обучения GAN                                      ║
║                                                                  ║
║  💡 Возможные применения:                                        ║
║     • Оценка качества генеративных моделей                       ║
║     • Дополнительный loss для улучшения реалистичности           ║
║     • Фильтрация сгенерированного контента                       ║
║     • Детекция deepfakes и AI-сгенерированных изображений        ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
""")

print(f"\n📊 Финальные метрики:")
print(f"   Начальный TIGAS(fake): {history['tigas_score_fake'][0]:.4f}")
print(f"   Финальный TIGAS(fake): {history['tigas_score_fake'][-1]:.4f}")
print(f"   Улучшение: +{history['tigas_score_fake'][-1] - history['tigas_score_fake'][0]:.4f}")
print(f"\n   TIGAS(real) baseline: {history['tigas_score_real'][-1]:.4f}")

## 🔗 Ссылки

- **GitHub**: [H1merka/TIGAS](https://github.com/H1merka/TIGAS)
- **HuggingFace**: [H1merka/TIGAS](https://huggingface.co/H1merka/TIGAS)
- **PyPI**: `pip install tigas-metric`

---

```python
# Быстрый старт
from tigas import TIGAS

tigas = TIGAS(auto_download=True)
score = tigas('your_image.jpg')  # 1.0 = real, 0.0 = fake

# Как loss для генератора
loss = 1.0 - tigas(generated_images).mean()
loss.backward()
```